In [1]:
from urllib.request import urlopen
from fastai.vision import *
from fastai.metrics import error_rate
from PIL import Image
#from google.colab import drive
from os import listdir
import numpy as np 
import cv2
from matplotlib import image
from matplotlib import pyplot
import numpy as np
import warnings
import pandas as pd
import os 
import torch
from fastai.vision.data import *
import fastai
from pathlib import Path
from shutil import copyfile
import random
import shutil
from PIL import Image
from datetime import datetime
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt import  Trials
import torch.nn as nn
from datetime import datetime
from time import mktime
import time
from sklearn.model_selection import RepeatedKFold

In [2]:
def get_float_label(x):
    y = str(x)[(len(path_str) + 1):(len(path_str) + 3)]
    dash_search = re.search("/", y)
    if dash_search != None:
        y = y[:dash_search.span()[0]]
    y = float(y)
    return y

In [3]:
path_str = "/media/revilo/oliver-kool-2/pawpularity_data/train_for_resnet"
path = Path(path_str)

warnings.filterwarnings('ignore')

class L1LossFlat(nn.L1Loss):
#“Mean Absolute Error Loss”
  def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
    return super().forward(input.view(-1), target.view(-1))


In [4]:

current_dir = ! pwd
current_dir = current_dir[0]

# Fastai trainable parameters for image processing and
# CNN

space = hp.choice('a',
    [('MSE loss', 
        hp.choice('size', [64, 128, 224]), # image size
        hp.uniform('weight_dec', 0, 0.99), # weight decay
         hp.uniform('div_factor', 1, 100), # div_factor
         hp.uniform('mom1',  0.5,0.85),  # momemtum1
         hp.uniform('mom2',  0.86, 0.99),  # momemtum2
         hp.uniform("max_rotate", 0, 360), # rotation angle
         hp.uniform("max_zoom", 1, 3), # max_zoom
         hp.uniform("p_affine", 0, 0.99), # p_affine,
         hp.uniform("max_lighting", 0, 0.99), # max_lighting
         hp.uniform("max_warp", 0, 0.99),# max warp 
        hp.choice("loss_fn", [MSELossFlat, L1LossFlat]),
        hp.choice("model", [models.resnet18, models.resnet34,
                           models.resnet50])
     )
    ])
                  

def rmse_basic(x, y):
    return np.sqrt(np.mean((x-y)**2))

## CV as part of objective function

In [5]:
# file names from train.sv

data_dir =  "/media/revilo/oliver-kool-2/pawpularity_data/"
df = pd.read_csv(f'{data_dir}/train.csv')
df.Id = df.Id.apply(lambda x: x + ".jpg")
df = df[["Id", "Pawpularity"]]#.rename(columns = {"Pawpularity":})
df["Id"] = df["Pawpularity"].astype(str) + "/" +  df["Id"]

In [6]:
#from sklearn.model_selection import RepeatedStratifiedKFold
def get_test_indices(df, no_of_folds = 5):
    #random_state = 12883823
    rkf = RepeatedKFold(n_splits=no_of_folds, 
                        n_repeats=1) #random_state=random_state
    test_indexes = []
    for train, test in rkf.split(df):
        test_indexes.append(test)
    return test_indexes

In [23]:
max_evals = 30
minutes_per_epoch = 1 # with GTX 1080 8GB and ryzen 5600x
no_of_epochs = 3
no_of_folds = 5
expected_training_time = max_evals*minutes_per_epoch*no_of_epochs*no_of_folds
print("Expected training time: " + str(expected_training_time) + " minutes")

Expected training time: 450 minutes


In [24]:
rmses_by_fold = []

def objective(args):
    (label,
     size,
    weight_dec, 
    div_factor,
    mom1, 
    mom2, 
    max_rotate,
    max_zoom,
    p_affine,
    max_lighting, 
    max_warp,
    loss_fn,
    model) = args
    
    test_indexes = get_test_indices(df, no_of_folds = no_of_folds)
    rmse_list = []
    
    for test_index_array in test_indexes:
        
        data = (ImageList.from_folder(path)\
            .split_by_idx(test_index_array)\
            .label_from_func(get_float_label, label_cls = 
               FloatList)\
        .transform(get_transforms(do_flip=False,
                                 max_rotate = max_rotate,
                                 max_zoom = max_zoom,
                                 p_affine = p_affine,
                                 max_lighting = max_lighting,
                                 max_warp = max_warp), 
                   size=size)\
        .databunch())

        data.normalize(imagenet_stats) 

        learn = cnn_learner(data, model, model_dir = Path(current_dir))
        learn.loss = loss_fn
        learn.fit_one_cycle(no_of_epochs,
                            div_factor = div_factor,
                            moms = (mom1, mom2),
                            wd = weight_dec)

        interp = ClassificationInterpretation.from_learner(learn)
        rmse_res = rmse_basic(interp.preds.numpy().reshape(-1), 
            interp.y_true.numpy()) 
        rmse_list.append(rmse_res)
        
        t1 = time.localtime()
        dt = datetime.fromtimestamp(mktime(t1))
    
        learn.save(f'resnet_models/{str(dt)}_{np.round(rmse_res, 2)}.pt')
        del learn
        torch.cuda.empty_cache()
    
    final_rmse = np.mean(rmse_list)
    rmses_by_fold.append(rmse_list)

    return final_rmse



In [25]:
print("Baseline RMSE to beat: 20.5851")
baseline_rmse = 20.5851

trials = Trials()
best = fmin(objective, space, algo=tpe.suggest, max_evals=max_evals,
           trials = trials)

losses = [x["loss"] for x in trials.results]

if losses[len(losses) - 1] < baseline_rmse:
    print("RMSE below baseline")

print(best)
print(space_eval(space, best))


Baseline RMSE to beat: 20.5851
  0%|          | 0/30 [00:00<?, ?trial/s, best loss=?]

epoch,train_loss,valid_loss,time
0,1237.831665,801.832886,00:47
1,445.484955,351.969086,00:47
2,351.529663,349.812347,00:47


epoch,train_loss,valid_loss,time
0,1249.951172,765.366272,00:47
1,452.815552,349.438538,00:47
2,357.096344,336.525024,00:47


epoch,train_loss,valid_loss,time
0,1254.945312,737.676208,00:47
1,449.080414,353.924744,00:47
2,357.568542,344.532654,00:47


epoch,train_loss,valid_loss,time
0,1247.722778,784.833618,00:47
1,451.721466,345.753815,00:47
2,361.347839,332.972260,00:47


...

epoch,train_loss,valid_loss,time
0,1241.861694,758.983582,00:57
1,430.955475,314.522125,00:57
2,331.058197,305.504669,00:57


epoch,train_loss,valid_loss,time
0,1237.895752,811.792542,00:58
1,436.270538,313.040588,00:58
2,329.834351,309.154968,00:57


epoch,train_loss,valid_loss,time
0,1253.467407,745.798157,00:57
1,433.758484,317.327942,00:57
2,332.138672,314.398529,00:57


epoch,train_loss,valid_loss,time
0,1258.184082,760.746704,00:57
1,431.102142,308.390289,00:58
2,327.440399,304.263245,00:57


100%|██████████| 30/30 [7:11:40<00:00, 863.35s/trial, best loss: 17.460851669311523]
RMSE below baseline
{'a': 0, 'div_factor': 1.6043067786085103, 'loss_fn': 0, 'max_lighting': 0.2391272797023367, 'max_rotate': 208.78583621669668, 'max_warp': 0.10510357270731724, 'max_zoom': 2.4716600773278135, 'model': 1, 'mom1': 0.7412407050621035, 'mom2': 0.9707736130395528, 'p_affine': 0.5694889423615632, 'size': 2, 'weight_dec': 0.3806114553011142}
('MSE loss', 224, 0.3806114553011142, 1.6043067786085103, 0.7412407050621035, 0.9707736130395528, 208.78583621669668, 2.4716600773278135, 0.5694889423615632, 0.2391272797023367, 0.10510357270731724, <function MSELossFlat at 0x7f28024e09d0>, <function resnet34 at 0x7f2801d9ef70>)


In [26]:
trials_df = pd.DataFrame(trials.trials)
trials_df_small = pd.DataFrame(trials.vals)
trials_df["RMSE"] = trials.losses()
trials_df_small["RMSE"] = trials.losses()

t1 = time.localtime()
dt = datetime.fromtimestamp(mktime(t1))

In [29]:
trials_df.to_csv(f'resnet_trials/trials_df_{dt}.csv')

In [30]:
trials_df_small.to_csv(f'resnet_trials/trials_df_small_{dt}.csv')